# Prediction of on time delivery of products

In [ ]:
#Importing required libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder , StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency



In [ ]:
#Reading the dataset
dataset = pd.read_csv('/kaggle/input/customer-analytics/Train.csv')
dataset.head()

# Data Profiling

In [ ]:
# Shape of the dataset
dataset.shape

In [ ]:
#Information about the columns
dataset.info()

In [ ]:
#Checking if there is any null values in the dataset
dataset.isnull().sum()

**Observations:**
* No missing values in the dataset

In [ ]:
#Basic summary statistics
dataset.describe()

# Exploratory Data Analysis

In [ ]:
dataset.head()

In [ ]:
delay=pd.DataFrame(dataset.groupby(['Reached.on.Time_Y.N'])['ID'].count()/len(dataset)).reset_index()
plt.pie(delay['ID'],labels=delay['Reached.on.Time_Y.N'],autopct='%1.2f%%')

**Observations:**
* 40% of the total deliveries are not reached on time

In [ ]:
data_v1 = dataset[dataset['Reached.on.Time_Y.N']==0]

In [ ]:
data_v2=pd.DataFrame((data_v1.groupby(['Mode_of_Shipment'])['ID'].count())/len(data_v1)*100)
data_v2=data_v2.reset_index()
visual=sns.barplot(x="Mode_of_Shipment", y="ID", data=data_v2 )
for index, row in data_v2.iterrows():
    visual.text(row.name,row.ID, round(row.ID,2), color='black', ha="center")
plt.title('Total number of delayed deliveries vs Shipment mode')
plt.ylabel('percentage')

**Observations:**
* Around 68% of the delayed deliveries are caused when ships are used as a mode of shipments. So, Alternate options like Flight and Road services might be considered to reduce the delayed deliveries 

In [ ]:
data_v3=pd.DataFrame((data_v1.groupby(['Warehouse_block'])['ID'].count())/len(data_v1)*100)
data_v3=data_v3.reset_index()
visual=sns.barplot(x="Warehouse_block", y="ID", data=data_v3 )
for index, row in data_v3.iterrows():
    visual.text(row.name,row.ID, round(row.ID,2), color='black', ha="center")
plt.title('Total number of delayed deliveries vs Warehouse block')
plt.ylabel('percentage')

**Observations**
* Higher percent of delayed deliveries are recorded in Warehouse block F. For rest of the block,the percent of delayed deliveries are almost consistent

In [ ]:
data_v4=pd.DataFrame((dataset.groupby(['Customer_rating'])['ID'].count())/len(dataset)*100)
data_v4=data_v4.reset_index()
visual=sns.barplot(x="Customer_rating", y="ID", data=data_v4 )
for index, row in data_v4.iterrows():
    visual.text(row.name,row.ID, round(row.ID,2), color='black', ha="center")
plt.title('Total percentages of different ratings')
plt.ylabel('percentage')

**Observations:**
* The percentage of different ratings given by customers seems to same. Almost 20% of the total deliveries received 5 ratings

In [ ]:
sns.countplot(x = "Product_importance", data = dataset, hue="Reached.on.Time_Y.N")
plt.title("Product importance by Reached on Time or Not")

**Observations:**
* Higher number of deliveries falls under low product importance. Very less number of highly important products delivered. It means customers are ordering more number of low importance products from this ecommerce group

In [ ]:
sns.countplot(x = "Gender", data = dataset, hue="Reached.on.Time_Y.N")
plt.title("Gender by Reached on Time or Not")

In [ ]:
sns.countplot(x = "Warehouse_block", data = dataset, hue="Reached.on.Time_Y.N")
plt.title("Warehouse block vs on Time or Not")

**Observations:**
* More number of deliveries were from Ware house block 'F'
* For rest of the blocks, the pattern remains same

In [ ]:
sns.countplot(x = "Customer_rating", data = dataset, hue="Gender")
plt.title("Customer ratings by Gender")

In [ ]:
sns.boxplot(x='Gender',y='Cost_of_the_Product',data=dataset,hue='Gender')
plt.title("Amount spent by each Gender")

**Observations**
* Both men and women seems to spend same level of amount on average while purchasing a product 

In [ ]:
dataset.groupby(['Product_importance'])['Cost_of_the_Product'].mean().plot.bar()
plt.title("Average cost of products of different importance")

**Observations:**
* The average value of higher importance is slightly lesser than both low and high importance products. So, the product importance is not assigned based on the monetory value
* Overall there isnt much variation in the features

In [ ]:
dataset.head()

In [ ]:
dataset[['Cost_of_the_Product','Prior_purchases','Discount_offered','Weight_in_gms']].hist()


**Observations**
* 'Discount_offered' is positively skewed
* Even 'prior purchase' is also positively skewed

In [ ]:
corr = dataset.corr()
sns.heatmap(corr, cmap="Blues", annot=True)

**Observations:**
*  There is a slight positive correlation between 'cost of products' and 'customer care calls'

# Data preparation

In [ ]:
#Dropping the ID column because of high cardinality
data=dataset.drop(['ID'],axis=1)
data.head()

**Chi square test of independence**

In [ ]:
#Warehouse_block
crossTab = pd.crosstab(data['Warehouse_block'], data['Reached.on.Time_Y.N'])
ChiSqResult = chi2_contingency(crossTab)
print("p-value ",ChiSqResult[1])

In [ ]:
#Warehouse_block
crossTab = pd.crosstab(data['Mode_of_Shipment'],data['Reached.on.Time_Y.N'])
ChiSqResult = chi2_contingency(crossTab)
print("p-value ",ChiSqResult[1])

In [ ]:
# Product_importance
crossTab = pd.crosstab(data['Product_importance'],data['Reached.on.Time_Y.N'])
ChiSqResult = chi2_contingency(crossTab)
print("p-value ",ChiSqResult[1])

In [ ]:
# Gender
crossTab = pd.crosstab(data['Gender'],data['Reached.on.Time_Y.N'])
ChiSqResult = chi2_contingency(crossTab)
print("p-value ",ChiSqResult[1])

**Observations:**
* From chi square test, we can conclude that three of the independent categorical features are not related to the response variable beacause the p value is greater than 0.05. So, these features can be removed and only product importance feature can be included

In [ ]:
#Renaming the column Reached.on.Time_Y.N
data.rename(columns={'Reached.on.Time_Y.N':'Reached on Time'}, inplace=True)

In [ ]:
data=pd.get_dummies(data,columns=['Product_importance'], drop_first=True)
data.head()

In [ ]:
X=data.drop(['Reached on Time','Warehouse_block','Mode_of_Shipment','Gender'],axis=1)
y=data['Reached on Time']

In [ ]:
#Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

#Scaling the data
ms = MinMaxScaler()
X_train = ms.fit_transform(X_train)
X_test = ms.fit_transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train,columns=X.columns)
X_test  =pd.DataFrame(X_test,columns=X.columns)
X_train.head()

**Support Vector Machine**

In [ ]:
svm_model = svm.SVC(gamma='auto',C=5,kernel='rbf')
svm_model.fit(X_train,y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test,y_pred))

**Random Forest Classifier**

In [ ]:
params = {'n_estimators':[100,150], 'criterion':['gini', 'entropy']}
#Hyper parameter tuning
rf_model =GridSearchCV(estimator=RandomForestClassifier(),param_grid=params,scoring='accuracy', cv=5)
rf_model = rf_model.fit(X_train,y_train)
y_pred=rf_model.predict(X_test)
print(classification_report(y_test,y_pred))

**Artificial Neural Network**

In [ ]:
ann = Sequential()
ann.add(Dense(14,input_dim=8,activation='relu'))
ann.add(Dense(8,activation='relu'))
ann.add(Dense(8,activation='relu'))
ann.add(Dense(1,activation='sigmoid'))
ann.compile(loss="binary_crossentropy", optimizer='SGD',metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=100, batch_size=15)

In [ ]:
predictions = (ann.predict(X_test) > 0.5)
print(classification_report(y_test,predictions))

**Conclusion:**
* ANN gives better accuracy of 67% than SVM and Random forest